In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import lazypredict
from lazypredict.Supervised import LazyClassifier
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

C:\Users\berna\AppData\Roaming\Python\Python38\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\Users\berna\anaconda3\envs\teachopencadd\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
# Set path to this notebook
HERE = Path(_dh[-1])
HDAC1and6 = Path(HERE).resolve().parents[1]/'input'
output = HERE/'OUTPUT'

In [3]:
df1 = pd.read_pickle(HDAC1and6/"HDAC1and6_SemiSel_1024B.csv")
df1 = df1.sample(frac=1).reset_index(drop=True)
df1.shape

(1910, 6)

In [4]:
# Add column for selectivity
df1["selectivity"] = np.zeros(len(df1))

# Mark every molecule as selective if SelectivityWindow is >=2 or >=-2, 0 otherwise
df1.loc[df1[df1.SelectivityWindow >= 2.0].index, "selectivity"] = 1.0
df1.loc[df1[df1.SelectivityWindow <= -2.0].index, "selectivity"] = 1.0
# NBVAL_CHECK_OUTPUT
print("Number of selective compounds:", int(df1.selectivity.sum()))
print("Number of nonselective compounds:", len(df1) - int(df1.selectivity.sum()))

Number of selective compounds: 334
Number of nonselective compounds: 1576


In [5]:
df1.head(5)

,molecule_chembl_id,fp_MACCS,fp_Morgan3,fp_MorganF,fp_MAP4,SelectivityWindow,selectivity
0,CHEMBL3953547,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, ...","[2331191, 3821889, 137380, 4488302, 7332227, 1...",1.75,0.00
1,CHEMBL4160522,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, ...","[9864737, 5343410, 5083874, 12198374, 1042149,...",2.61,1.00
2,CHEMBL479602,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, ...","[1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[6670835, 5451723, 2175494, 2772628, 1323697, ...",0.35,0.00
3,CHEMBL3526395,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[6014252, 13528188, 4046732, 242486, 27304819,...",0.36,0.00
4,CHEMBL3655951,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, ...","[35600637, 2478511, 10872982, 5588261, 1206207...",2.17,1.00


#### By using MACCS fingerprints that are 166 bits 

In [6]:
X1 = np.array(list((df1['fp_MACCS']))).astype(float)
#X.shape
Y1 = df1["selectivity"].values
#y.shape

In [7]:
# Split the data into training and test set
# Perform data splitting using 80/20 ratio
X_train1, X_test1, Y_train1, Y_test1 = train_test_split(X1, Y1, test_size=0.2, random_state=42)

# Print the shape of training and testing data
print("Shape of training data:", X_train1.shape)
print("Shape of test data:", X_test1.shape)
# NBVAL_CHECK_OUTPUT

Shape of training data: (1528, 167)
Shape of test data: (382, 167)


In [8]:
# Defines and builds the LazyClassifier
clf1 =  LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
model1,predictions1 = clf1.fit(X_train1, X_test1, Y_train1, Y_test1)

 90%|█████████████████████████████████████████████████████████████████████████▌        | 26/29 [00:04<00:00,  7.36it/s]

[20:11:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:05<00:00,  5.50it/s]


In [9]:
predictions1.sort_values(by="F1 Score", ascending=False).head(10)

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LGBMClassifier,0.87,0.75,0.75,0.86,0.25
XGBClassifier,0.87,0.75,0.75,0.86,0.47
ExtraTreeClassifier,0.86,0.77,0.77,0.85,0.03
BaggingClassifier,0.86,0.73,0.73,0.85,0.16
ExtraTreesClassifier,0.86,0.73,0.73,0.85,0.30
LinearSVC,0.86,0.72,0.72,0.85,0.37
DecisionTreeClassifier,0.85,0.74,0.74,0.84,0.05
RandomForestClassifier,0.85,0.72,0.72,0.84,0.34
SVC,0.86,0.69,0.69,0.84,0.31


### USING 1024 BITS FINGERPRINTS

In [10]:
#By using Morgan fingerprints with radius of 3 and 1024 bits
X2 = np.array(list((df1['fp_Morgan3']))).astype(float)
#X.shape
Y2 = df1["selectivity"].values
#y.shape

In [11]:
# Split the data into training and test set
# Perform data splitting using 80/20 ratio
X_train2, X_test2, Y_train2, Y_test2 = train_test_split(X2, Y2, test_size=0.2, random_state=42)

# Print the shape of training and testing data
print("Shape of training data:", X_train2.shape)
print("Shape of test data:", X_test2.shape)
# NBVAL_CHECK_OUTPUT

Shape of training data: (1528, 1024)
Shape of test data: (382, 1024)


In [12]:
clf2 = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models2, predictions2 = clf2.fit(X_train2, X_test2, Y_train2, Y_test2)

 90%|█████████████████████████████████████████████████████████████████████████▌        | 26/29 [00:22<00:03,  1.07s/it]

[20:11:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:24<00:00,  1.18it/s]


In [13]:
predictions2.sort_values(by="F1 Score", ascending=False).head(10)

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
ExtraTreesClassifier,0.89,0.78,0.78,0.88,0.82
RandomForestClassifier,0.88,0.76,0.76,0.87,0.68
LGBMClassifier,0.87,0.75,0.75,0.86,0.74
XGBClassifier,0.87,0.76,0.76,0.86,1.49
BaggingClassifier,0.87,0.74,0.74,0.86,1.34
SVC,0.87,0.72,0.72,0.86,2.55
KNeighborsClassifier,0.87,0.74,0.74,0.86,1.48
AdaBoostClassifier,0.86,0.73,0.73,0.85,1.66
ExtraTreeClassifier,0.84,0.76,0.76,0.84,0.11


In [14]:
#By using Morgan fingerprints with radius of 3 and 1024 bits and feautures
X3 = np.array(list((df1['fp_MorganF']))).astype(float)
#X.shape
Y3 = df1["selectivity"].values
#y.shape

In [15]:
# Split the data into training and test set
# Perform data splitting using 80/20 ratio
X_train3, X_test3, Y_train3, Y_test3 = train_test_split(X3, Y3, test_size=0.2, random_state=42)

# Print the shape of training and testing data
print("Shape of training data:", X_train3.shape)
print("Shape of test data:", X_test3.shape)
# NBVAL_CHECK_OUTPUT

Shape of training data: (1528, 1024)
Shape of test data: (382, 1024)


In [16]:
clf3 = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models3, predictions3 = clf3.fit(X_train3, X_test3, Y_train3, Y_test3)

 90%|█████████████████████████████████████████████████████████████████████████▌        | 26/29 [00:23<00:03,  1.02s/it]

[20:12:25] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:25<00:00,  1.14it/s]


In [17]:
predictions3.sort_values(by="F1 Score", ascending=False).head(10)

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
BaggingClassifier,0.89,0.76,0.76,0.88,1.30
ExtraTreesClassifier,0.88,0.76,0.76,0.87,0.86
XGBClassifier,0.88,0.75,0.75,0.87,1.48
RandomForestClassifier,0.87,0.74,0.74,0.86,0.75
SVC,0.87,0.73,0.73,0.86,2.38
LGBMClassifier,0.87,0.74,0.74,0.86,0.67
DecisionTreeClassifier,0.86,0.75,0.75,0.86,0.38
AdaBoostClassifier,0.86,0.73,0.73,0.85,2.08
Perceptron,0.85,0.75,0.75,0.84,0.19


In [18]:
#By using MAP4 fingerprints with 1024 bits
X4 = np.array(list((df1['fp_MAP4']))).astype(float)
#X.shape
Y4 = df1["selectivity"].values
#y.shape

In [19]:
# Split the data into training and test set
# Perform data splitting using 80/20 ratio
X_train4, X_test4, Y_train4, Y_test4 = train_test_split(X4, Y4, test_size=0.2, random_state=42)

# Print the shape of training and testing data
print("Shape of training data:", X_train4.shape)
print("Shape of test data:", X_test4.shape)
# NBVAL_CHECK_OUTPUT

Shape of training data: (1528, 1024)
Shape of test data: (382, 1024)


In [20]:
# Defines and builds the lazyclassifier
clf4 = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models4, predictions4 = clf4.fit(X_train4, X_test4, Y_train4, Y_test4)

 90%|█████████████████████████████████████████████████████████████████████████▌        | 26/29 [00:46<00:03,  1.20s/it]

[20:13:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [01:00<00:00,  2.08s/it]


In [21]:
predictions4.sort_values(by="F1 Score", ascending=False).head(10)

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LGBMClassifier,0.88,0.76,0.76,0.88,9.72
XGBClassifier,0.88,0.76,0.76,0.87,4.05
RandomForestClassifier,0.88,0.73,0.73,0.86,3.19
KNeighborsClassifier,0.87,0.74,0.74,0.86,1.50
BaggingClassifier,0.87,0.72,0.72,0.86,16.18
AdaBoostClassifier,0.87,0.72,0.72,0.85,7.00
ExtraTreesClassifier,0.86,0.69,0.69,0.84,0.87
DecisionTreeClassifier,0.84,0.74,0.74,0.84,2.17
PassiveAggressiveClassifier,0.84,0.73,0.73,0.84,0.32


## USE 2048 BITS FINGERPRINTS

In [22]:
df2 = pd.read_pickle(HDAC1and6/"HDAC1and6_SemiSel_1024B.csv")
df2 = df2.sample(frac=1).reset_index(drop=True)
df2

,molecule_chembl_id,fp_MACCS,fp_Morgan3,fp_MorganF,fp_MAP4,SelectivityWindow
0,CHEMBL470470,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[21551503, 888267, 7547857, 10808442, 53812731...",0.60
1,CHEMBL4464975,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, ...","[569161, 3913535, 6718756, 2817337, 6615052, 6...",2.74
2,CHEMBL1934897,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, ...","[8292377, 3821889, 526260, 9391761, 235748, 73...",-0.20
3,CHEMBL1767045,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, ...","[5196932, 783386, 4442399, 6897825, 553789, 11...",0.10
4,CHEMBL4539040,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, ...","[12058776, 3008293, 310797, 7125875, 638552, 1...",2.14
...,...,...,...,...,...,...
1905,CHEMBL3972758,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, ...","[4712070, 2478511, 10872982, 11123208, 1422872...",2.08
1906,CHEMBL482095,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, ...","[21389803, 23756935, 2078754, 35909301, 237981...",0.44
1907,CHEMBL3655959,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, ...","[13179826, 2478511, 10872982, 61527453, 480391...",0.90
1908,CHEMBL3655963,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, ...","[35600637, 2478511, 6486141, 5588261, 1206207,...",1.79


In [23]:
# Add column for selectivity
df2["selectivity"] = np.zeros(len(df2))

# Mark every molecule as selective if SelectivityWindow is >=2 or >=-2, 0 otherwise
df2.loc[df2[df2.SelectivityWindow >= 2.0].index, "selectivity"] = 1.0
df2.loc[df2[df2.SelectivityWindow <= -2.0].index, "selectivity"] = 1.0
# NBVAL_CHECK_OUTPUT
print("Number of selective compounds:", int(df2.selectivity.sum()))
print("Number of nonselective compounds:", len(df2) - int(df2.selectivity.sum()))

Number of selective compounds: 334
Number of nonselective compounds: 1576


In [24]:
#2048 bits Morgan fingerprints with radius of 3
X5 = np.array(list((df2['fp_Morgan3']))).astype(float)
#X.shape
Y5 = df2["selectivity"].values
#y.shape

In [25]:
# Split the data into training and test set
# Perform data splitting using 80/20 ratio
X_train5, X_test5, Y_train5, Y_test5 = train_test_split(X5, Y5, test_size=0.2, random_state=42)

# Print the shape of training and testing data
print("Shape of training data:", X_train5.shape)
print("Shape of test data:", X_test5.shape)
# NBVAL_CHECK_OUTPUT

Shape of training data: (1528, 1024)
Shape of test data: (382, 1024)


In [26]:
clf5 = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models5, predictions5 = clf5.fit(X_train5, X_test5, Y_train5, Y_test5)

 90%|█████████████████████████████████████████████████████████████████████████▌        | 26/29 [00:22<00:03,  1.06s/it]

[20:13:50] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:25<00:00,  1.15it/s]


In [27]:
predictions5.sort_values(by="F1 Score", ascending=False).head(10)

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LGBMClassifier,0.91,0.82,0.82,0.91,0.90
RandomForestClassifier,0.91,0.79,0.79,0.90,0.69
ExtraTreesClassifier,0.90,0.77,0.77,0.89,0.85
BaggingClassifier,0.89,0.77,0.77,0.89,1.34
KNeighborsClassifier,0.89,0.76,0.76,0.88,1.48
XGBClassifier,0.89,0.75,0.75,0.88,1.82
Perceptron,0.87,0.82,0.82,0.88,0.20
ExtraTreeClassifier,0.88,0.77,0.77,0.88,0.11
AdaBoostClassifier,0.87,0.75,0.75,0.86,1.64


In [28]:
#2048 bits Morgan fingerprints with feautures
X6 = np.array(list((df2['fp_MorganF']))).astype(float)
#X.shape
Y6 = df2["selectivity"].values
#y.shape

In [29]:
# Split the data into training and test set
# Perform data splitting using 80/20 ratio
X_train6, X_test6, Y_train6, Y_test6 = train_test_split(X6, Y6, test_size=0.2, random_state=42)

# Print the shape of training and testing data
print("Shape of training data:", X_train6.shape)
print("Shape of test data:", X_test6.shape)
# NBVAL_CHECK_OUTPUT

Shape of training data: (1528, 1024)
Shape of test data: (382, 1024)


In [30]:
clf6 = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models6, predictions6 = clf6.fit(X_train6, X_test6, Y_train6, Y_test6)

 90%|█████████████████████████████████████████████████████████████████████████▌        | 26/29 [00:23<00:03,  1.03s/it]

[20:14:17] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:26<00:00,  1.09it/s]


In [31]:
predictions6.sort_values(by="F1 Score", ascending=False).head(10)

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
XGBClassifier,0.89,0.77,0.77,0.89,1.96
RandomForestClassifier,0.89,0.75,0.75,0.88,0.72
BaggingClassifier,0.89,0.75,0.75,0.88,1.42
ExtraTreesClassifier,0.89,0.76,0.76,0.88,0.91
LGBMClassifier,0.88,0.77,0.77,0.88,1.32
Perceptron,0.86,0.79,0.79,0.86,0.18
DecisionTreeClassifier,0.87,0.73,0.73,0.86,0.33
KNeighborsClassifier,0.86,0.73,0.73,0.86,1.46
AdaBoostClassifier,0.86,0.72,0.72,0.85,1.66


In [32]:
#2048 bits MAP4 fingerprints
X7 = np.array(list((df2['fp_MAP4']))).astype(float)
#X.shape
Y7 = df2["selectivity"].values
#y.shape

In [33]:
# Split the data into training and test set
# Perform data splitting using 80/20 ratio
X_train7, X_test7, Y_train7, Y_test7 = train_test_split(X7, Y7, test_size=0.2, random_state=42)

# Print the shape of training and testing data
print("Shape of training data:", X_train7.shape)
print("Shape of test data:", X_test7.shape)
# NBVAL_CHECK_OUTPUT

Shape of training data: (1528, 1024)
Shape of test data: (382, 1024)


In [34]:
# Defines and builds the lazyclassifier
clf7 = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models7, predictions7 = clf7.fit(X_train7, X_test7, Y_train7, Y_test7)

 90%|█████████████████████████████████████████████████████████████████████████▌        | 26/29 [00:46<00:03,  1.22s/it]

[20:15:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [01:02<00:00,  2.16s/it]


In [35]:
predictions7.sort_values(by="F1 Score", ascending=False).head(10)

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LGBMClassifier,0.90,0.76,0.76,0.89,11.42
XGBClassifier,0.89,0.74,0.74,0.88,4.66
AdaBoostClassifier,0.88,0.73,0.73,0.87,6.34
ExtraTreesClassifier,0.88,0.70,0.70,0.86,0.89
Perceptron,0.87,0.75,0.75,0.86,0.20
KNeighborsClassifier,0.87,0.73,0.73,0.86,1.50
BaggingClassifier,0.88,0.70,0.70,0.86,15.80
PassiveAggressiveClassifier,0.86,0.76,0.76,0.86,0.24
SGDClassifier,0.87,0.71,0.71,0.86,0.20


In [36]:
with pd.ExcelWriter(output/"LazyPredictResults_SemiSelAdded.xlsx") as writer:
   
    # use to_excel function and specify the sheet_name and index 
    # to store the dataframe in specified sheet
    predictions1.to_excel(writer, sheet_name="MACCS", )
    predictions2.to_excel(writer, sheet_name="Morgan 1024 Bits", )
    predictions3.to_excel(writer, sheet_name="MorganF 1024 Bits", )
    predictions4.to_excel(writer, sheet_name="MAP4 1024 Bits", )
    predictions5.to_excel(writer, sheet_name="Morgan 2048 Bits", )
    predictions6.to_excel(writer, sheet_name="MorganF 2048 Bits", )
    predictions7.to_excel(writer, sheet_name="MAP4 2048 Bits", )
    